Calculates JJA zonal mean jet latitude values for 1979-2014 and 2070-2099 for CMIP6 models.  Outputs both the jet latitude of the ensemble mean zonal mean zonal wind and the jet latitude for the first ensemle member.  This code works on a CMIP archive that is internal to the Climate and Global Dynamics Laboratory, NCAR.  If you want to run this on your own archive, you'd need to modify histpath and rcp85path and make sure the directory structure is set up in the right way.

Output is saved to ecpaper2020/DATASORT/JLAT/DATA/jlatcmip6.nc

In [3]:
import importlib
import pandas as pd 
import xarray as xr
import numpy as np
from numpy import nan
import sys
import warnings
import math
from glob import glob

from ecpaper_utils import readdata_utils as read
from ecpaper_utils import jlat_utils as jlat
from ecpaper_utils import calendar_utils as cal

importlib.reload(read)
importlib.reload(jlat)
importlib.reload(cal)

warnings.filterwarnings('ignore')

Set paths for CMIP6 models (historical and SSP5-8.5) and variable to use and the pressure level to use (in Pa)

In [4]:
histpath="/project/cmip6/historical/Amon/"
rcp85path="/project/cmip6/ssp585/Amon/"
var="ua"
plevuse="70000"
pathout="./DATA/"

Information on the models being used is provided in cmip5csvinfo.csv.  This contains information on the models, number of members and whether there's any special order for the member numbers i.e., if they don't simply go from 1 to N.  Read in this info and set up the dates for each period.

In [5]:
cmip6models=pd.read_csv('../cmip6csvinfo.csv')
print(cmip6models)

ybegp = 1979 ; monbegp = 1 ; yendp = 2014 ; monendp = 12 # dates for Past period
ybegf = 2070 ; monbegf = 1 ; yendf = 2099 ; monendf = 12 # dates for Future period

# total number of months (used for checking)
nmonthsp = (yendp-ybegp-1)*12 + (12-monbegp+1) + monendp
nmonthsf = (yendf-ybegf-1)*12 + (12-monbegf+1) + monendf

# set up date names
datebegp=str(ybegp)+"-"+str(monbegp).zfill(2)
dateendp=str(yendp)+"-"+str(monendp).zfill(2)
datebegf=str(ybegf)+"-"+str(monbegf).zfill(2)
dateendf=str(yendf)+"-"+str(monendf).zfill(2)

              Model  Nmempast  Nmemfuture  ftypep  ftypef  \
0        ACCESS-CM2         2           1       1       1   
1     ACCESS-ESM1-5         3           3       1       1   
2     AWI-CM-1-1-MR         5           1       1       1   
3       BCC-CSM2-MR         3           1       1       1   
4       CAMS-CSM1-0         1           2       1       1   
5           CanESM5        25          25       1       1   
6     CanESM5-CanOE         3           3       1       1   
7             CESM2        10           2       1       1   
8       CESM2-WACCM         3           1       1       1   
9             CIESM         3           1       1       1   
10     CMCC-CM2-SR5         1           1       1       1   
11       CNRM-CM6-1        15           6       2       2   
12    CNRM-CM6-1-HR         1           1       2       2   
13      CNRM-ESM2-1         5           5       2       2   
14        EC-Earth3        10           7       1       1   
15    EC-Earth3-Veg     

This is the main part of the script.  It loops over models and ensemble members and calculates the zonal mean zonal wind at 700hPa for JJA.  This is then interpolated onto a 1degree latitude grid and the jet latitude is calculated for both the ensemble mean and the first member.  These are then output to netcdf.

In [7]:
models=cmip6models['Model']
nmodels=models.size

# initialize member 1 and ensemble mean jet latitude arrays
jlat1memp = xr.DataArray(np.zeros(nmodels), coords=[models], dims="model", name="jlat1memp")
jlat1memf = xr.DataArray(np.zeros(nmodels), coords=[models], dims="model", name="jlat1memf")
jlatemp = xr.DataArray(np.zeros(nmodels), coords=[models], dims="model", name="jlatemp")
jlatemf = xr.DataArray(np.zeros(nmodels), coords=[models], dims="model", name="jlatemf")


# loop over models
for index, modname in models.iteritems():
    
    # --- sort out the PAST period ---
    nmems=cmip6models.loc[index, "Nmempast"]
    ftype=cmip6models.loc[index, "ftypep"]
    ptype=cmip6models.loc[index, "ptype"]
    
    for imem in range(1, nmems+1, 1):
        if (math.isnan(ptype)):
            memstr="r"+str(imem)+"i1p1f"+str(int(ftype))
        else :
            memstr="r"+str(imem)+"i1p"+str(int(ptype))+"f"+str(int(ftype))
            
        # check if a special order is needed
        changeorder=cmip6models.loc[index, "specialorderpast"]
        if (type(changeorder) == str):
            changeordernp = np.array(changeorder.split(","))
            if (math.isnan(ptype)):
                memstr = "r"+str(changeordernp[imem-1])+"i1p1f"+str(int(ftype))
            else:
                memstr = "r"+str(changeordernp[imem-1])+"i1p"+str(int(ptype))+"f"+str(int(ftype))
        
        print("Processing past "+modname+" "+memstr+"...")
        
        histdir = glob(histpath+var+"/"+modname+"/"+memstr+"/*/")
        histdir = histdir[0]
        
        # read in zonal mean u for 1979-2014
        u = read.read_zonalmean_1lev(histdir+"*.nc", datebegp, dateendp, plevuse)
        
        # check if the right number of months are there
        if (u.time.size != nmonthsp):
            print("something's wring, nmonthsp="+str(nmonthsp)+" but u has size "+str(u.time.size))
            
        # calculate JJA mean and interpolate
        ujja = cal.season_mean(u, "ua", season="JJA")
        ujja = ujja.dropna('lat')
        ujjainterp = ujja.interp(lat=np.linspace(-90,90,181), method="cubic")
        
        # calculate the jet latitude for 1 member.  Initialize ensemble mean
        if (imem == 1):
            uem = xr.DataArray(np.zeros(ujjainterp["lat"].size), dims=["lat"], coords=[ujjainterp["lat"]])
            jlatv, jspeedv = jlat.calcjetlat(ujjainterp, -80, -20)
            jlat1memp[index] = jlatv
            
        uem[:] = uem[:] + ujjainterp[:]/float(nmems) # calculate ensemble mean
        
    jlatv, jspeed = jlat.calcjetlat( uem, -80, -20)
    jlatemp[index] = jlatv

    # --- end sort out the past ---
    
    # --- sort out the future ---
    nmems=cmip6models.loc[index, "Nmemfuture"]
    ftype=cmip6models.loc[index, "ftypef"]
    for imem in range(1, nmems+1, 1): # loop over members
        if (math.isnan(ptype)):
            memstr="r"+str(imem)+"i1p1f"+str(int(ftype))
        else:
            memstr="r"+str(imem)+"i1p"+str(int(ptype))+"f"+str(int(ftype))
        
        #check if a special order is needed
        changeorder=cmip6models.loc[index,"specialorderfuture"]
        if (type(changeorder) == str):
            changeordernp = np.array(changeorder.split(","))
            if (math.isnan(ptype)):
                memstr="r"+str(changeordernp[imem-1])+"i1p1f"+str(int(ftype))
            else:
                memstr="r"+str(changeordernp[imem-1])+"i1p"+str(int(ptype))+"f"+str(int(ftype))
                
        print("Processing future for "+modname+" "+memstr+"...")
        rcp85dir=glob(rcp85path+var+"/"+modname+"/"+memstr+"/*/")
        rcp85dir=rcp85dir[0]
        u=read.read_zonalmean_1lev(rcp85dir+"*.nc", datebegf,dateendf,plevuse)
        
        # check the size
        if (u.time.size != nmonthsf):
            print("something's wrong, nmonthsf="+str(nmonthsf)+" but u has size "+str(u.time.size))
            sys.exit("u for future for "+modname+" "+memstr+" doesn't have the right size")
        
        ujja = cal.season_mean(u, "ua", season="JJA") # calculate the JJA mean
        ujja = ujja.dropna('lat')
        ujjainterp = ujja.interp(lat=np.linspace(-90,90,181), method="cubic")
        
        # calculate the jet latitude for 1 member.  Initialize ensemble mean
        if (imem == 1):
            uem = xr.DataArray(np.zeros(ujjainterp["lat"].size), dims=["lat"], coords=[ujjainterp["lat"]])
            jlatv, jspeedv = jlat.calcjetlat(ujjainterp, -80, -20)
            jlat1memf[index]=jlatv
            
        uem[:]=uem[:]+ujjainterp[:]/float(nmems) # calculate ensembl emean
    
    jlatv, jspeedv = jlat.calcjetlat(uem, -80, -20)
    jlatemf[index] = jlatv
    # --- end sort out the future ----
                               
# output to netcdf 
    
jlatemp.to_netcdf(path=pathout+"jlatcmip6.nc")
jlatemf.to_netcdf(path=pathout+"jlatcmip6.nc", mode="a")
jlat1memp.to_netcdf(path=pathout+"jlatcmip6.nc", mode="a")
jlat1memf.to_netcdf(path=pathout+"jlatcmip6.nc", mode="a")
    
print("*********************DONE***********************")
    

Processing past ACCESS-CM2 r1i1p1f1...
Processing past ACCESS-CM2 r2i1p1f1...
Processing future for ACCESS-CM2 r1i1p1f1...
Processing past ACCESS-ESM1-5 r1i1p1f1...
Processing past ACCESS-ESM1-5 r2i1p1f1...
Processing past ACCESS-ESM1-5 r3i1p1f1...
Processing future for ACCESS-ESM1-5 r1i1p1f1...
Processing future for ACCESS-ESM1-5 r2i1p1f1...
Processing future for ACCESS-ESM1-5 r3i1p1f1...
Processing past AWI-CM-1-1-MR r1i1p1f1...
Processing past AWI-CM-1-1-MR r2i1p1f1...
Processing past AWI-CM-1-1-MR r3i1p1f1...
Processing past AWI-CM-1-1-MR r4i1p1f1...
Processing past AWI-CM-1-1-MR r5i1p1f1...
Processing future for AWI-CM-1-1-MR r1i1p1f1...
Processing past BCC-CSM2-MR r1i1p1f1...
Processing past BCC-CSM2-MR r2i1p1f1...
Processing past BCC-CSM2-MR r3i1p1f1...
Processing future for BCC-CSM2-MR r1i1p1f1...
Processing past CAMS-CSM1-0 r1i1p1f1...
Processing future for CAMS-CSM1-0 r1i1p1f1...
Processing future for CAMS-CSM1-0 r2i1p1f1...
Processing past CanESM5 r1i1p2f1...
Processing past

In [14]:
print(ujja)

<xarray.DataArray 'ua' (lat: 144)>
dask.array<mean_agg-aggregate, shape=(144,), dtype=float32, chunksize=(144,), chunktype=numpy.ndarray>
Coordinates:
    plev     float64 7e+04
  * lat      (lat) float64 -89.38 -88.12 -86.88 -85.62 ... 86.88 88.12 89.38
